In [8]:
import os
import pandas as pd
from glob import glob
import textstat
from utils import uniqueterms, ttr, mtld, hdd

# FP_CNN = '../../readable_news_lf/cnn_cc/'
FP_CNN = 'raw_data/cnn_cc/'
files = glob(os.path.join(FP_CNN, '*.zip'))
files.append('raw_data/cnn_cc/cnn-7.csv.gz')
files

['raw_data/cnn_cc/cnn-6.zip',
 'raw_data/cnn_cc/cnn-1.zip',
 'raw_data/cnn_cc/cnn-3.zip',
 'raw_data/cnn_cc/cnn-4.zip',
 'raw_data/cnn_cc/cnn-2.zip',
 'raw_data/cnn_cc/cnn-5.zip',
 'raw_data/cnn_cc/cnn-7.csv.gz']

In [12]:
_usecols = ['program.name', 'year', 'month', 'date', 'time', 'timezone', 'subhead', 'text']
df = (pd.concat([pd.read_csv(file, usecols=_usecols) for file in files])
      .query('text==text')
      .reset_index(drop=True)
      .assign(
          # Compute readability for text
          flesch_reading_ease=lambda df: df.text.apply(textstat.flesch_reading_ease),
          flesch_kincaid_grade=lambda df: df.text.apply(textstat.flesch_kincaid_grade),
          fog=lambda df: df.text.apply(textstat.gunning_fog), 
          smog=lambda df: df.text.apply(textstat.smog_index),  
          terms=lambda df: df.text.apply(textstat.lexicon_count),
          # Compute lexical richness for text
          uniqueterms=lambda df: df.text.apply(uniqueterms), 
          ttr=lambda df: df.text.apply(ttr),
          mtld=lambda df: df.text.apply(mtld),
          hdd=lambda df: df.text.apply(hdd),    
      )      
     )

In [13]:
(df
 .drop(['text'], axis=1)
 .to_csv('data/cnn_2000_2021_textstat.csv.gz', index=False, compression="gzip")
)